In [1]:
! pip install git+https://github.com/johnwmillr/LyricsGenius.git

  Cloning https://github.com/johnwmillr/LyricsGenius.git to c:\users\jessi\appdata\local\temp\pip-req-build-q90evpb4
  Resolved https://github.com/johnwmillr/LyricsGenius.git to commit 486fd52b81325c49006b1f4ab3130cd890963f9f
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/johnwmillr/LyricsGenius.git 'C:\Users\jessi\AppData\Local\Temp\pip-req-build-q90evpb4'

[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from bs4 import BeautifulSoup
import re
import lyricsgenius
import requests
from pathlib import Path
import pandas as pd
import string

In [3]:
client_access_token = 'JCgEmwjGOu1MfiulRcUpWz2rrtOSPPVf7S5-BgyOkf9SEVEkKxkB7jYiTMl1z9fV'
LyricsGenius = lyricsgenius.Genius(client_access_token)

In [93]:
# function to clean song titles
def clean_up(song_title):
    for punctuation in string.punctuation:
        if punctuation == '?' or punctuation == '!' or punctuation == '"' or punctuation == '.':
            # remove certain punctuation marks
            song_title = song_title.replace(punctuation, '')
    
    if 'bonus track' in song_title:
        # include bonus tracks
        before_bt_pattern = re.compile('.*(?=\(bonus track)')
        song_title_before_bt = before_bt_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_bt.strip()
        clean_song_title = clean_song_title.replace('/', '-')
    elif 'Feat' in song_title:
        # include songs with featuring artists
        before_feat_pattern = re.compile('.*(?=\(Feat)')
        song_title_before_feat = before_feat_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_feat.strip()
        clean_song_title = clean_song_title.replace('/', '-')
    elif 'feat' in song_title:
        # include songs with featuring artists
        before_feat2_pattern = re.compile('.*(?=\(feat)')
        song_title_before_feat2 = before_feat2_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_feat2.strip()
        clean_song_title = clean_song_title.replace('/', '-')
    elif 'Ft' in song_title:
        # include songs with featuring artists
        before_ft_pattern = re.compile('.*(?=\(Ft)')
        song_title_before_ft = before_ft_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_ft.strip()
        clean_song_title = clean_song_title.replace('/', '-')    
    else:
        song_title_no_lyrics = song_title.replace('Lyrics', '')
        clean_song_title = song_title_no_lyrics.strip()
        clean_song_title = clean_song_title.replace('/', '-')
    
    return clean_song_title

# function to filter through which songs to keep
def songs_to_include(song_titles):
    clean_songs = []
    
    for song_title in song_titles:
        if 'Taylor\'s Version' in song_title: # and not 'From the Vault' in song_title:
            # only include TV only if From the Vault
            if 'From the Vault' in song_title or 'From The Vault' in song_title:
                clean_song = clean_up(song_title)
                clean_songs.append(clean_song)  
            else:
                pass
        elif 'Voice Memo' in song_title or 'Prologue' in song_title or 'A Message From Taylor' in song_title:
            pass  # do not include
        elif 'Original Demo Recording' in song_title or 'Poem' in song_title or 'Foreword' in song_title:
            pass  # do not include
        elif 'Acoustic' in song_title or 'Acoustic Version' in song_title or '(Piano Version)' in song_title:
            pass  # do not include
        elif 'Pop Mix' in song_title or 'Pop Version' in song_title or 'Remix' in song_title:
            pass  # do not include
        else:
            clean_song = clean_up(song_title)
            clean_songs.append(clean_song)
    
    return clean_songs

# function to retrieve all songs by artist from given albums from Genius
def get_all_songs_from_album(artist, album_name):
    # replace spaces in artist and album names with dashes ('-')
    artist = artist.replace(' ', '-')
    album_name = album_name.replace(' ', '-')
    
    response = requests.get(f'https://genius.com/albums/{artist}/{album_name}')
    html_string = response.text
    document = BeautifulSoup(html_string, 'html.parser')

    song_title_tags = document.find_all('h3', attrs={'class': 'chart_row-content-title'})
    song_titles = [song_title.text for song_title in song_title_tags]
    included_songs = songs_to_include(song_titles)
               
    return included_songs

# function to download lyrics for all songs in album
def download_album_lyrics(artist, album_name):
    LyricsGenius.remove_section_headers = True
    
    # call get_all_songs_from_album function
    clean_songs = get_all_songs_from_album(artist, album_name)
    
    for song in clean_songs:
        song_object = LyricsGenius.search_song(song, artist)
        
        if song_object != None:
            artist_title = artist.replace(' ', '-')
            album_title = album_name.replace(' ', '-')
            song_title = song.replace('/', '-')
            song_title = song.replace(' ', '-')
            
            # create filename for each song inside a directory
            custom_filename = f'{artist_title}_{album_title}/{song_title}'
            Path(f'{artist_title}_{album_title}').mkdir(parents=True, exist_ok=True)
            
            # save lyrics as a txt files
            song_object.save_lyrics(filename=custom_filename, extension='txt', sanitize=False)
        else:
            # if song does not contain lyrics
            print('No lyrics')

In [19]:
# download_album_lyrics('Taylor Swift', 'Taylor Swift')

# download_album_lyrics('Taylor Swift', 'Fearless Platinum Edition')
# download_album_lyrics('Taylor Swift', 'Fearless Taylors Version')

# download_album_lyrics('Taylor Swift', 'Speak Now Deluxe Edition')
# download_album_lyrics('Taylor Swift', 'Speak Now Taylors Version')

# download_album_lyrics('Taylor Swift', 'Red Deluxe Edition')
# download_album_lyrics('Taylor Swift', 'Red Taylors Version')

# download_album_lyrics('Taylor Swift', '1989 Deluxe')
# download_album_lyrics('Taylor Swift', '1989 Taylors Version')

# download_album_lyrics('Taylor Swift', 'Reputation')

# download_album_lyrics('Taylor Swift', 'Lover')

# download_album_lyrics('Taylor Swift', 'folklore deluxe version')

# download_album_lyrics('Taylor Swift', 'evermore deluxe version')

# download_album_lyrics('Taylor Swift', 'Midnights The Til Dawn Edition')

# download_album_lyrics('Taylor Swift', 'The Tortured Poets Department The Anthology')

Searching for "Fortnight" by Taylor Swift...
Done.
Wrote Taylor-Swift_The-Tortured-Poets-Department-The-Anthology/Fortnight.txt.
Searching for "The Tortured Poets Department" by Taylor Swift...
Done.
Wrote Taylor-Swift_The-Tortured-Poets-Department-The-Anthology/The-Tortured-Poets-Department.txt.
Searching for "My Boy Only Breaks His Favorite Toys" by Taylor Swift...
Done.
Wrote Taylor-Swift_The-Tortured-Poets-Department-The-Anthology/My-Boy-Only-Breaks-His-Favorite-Toys.txt.
Searching for "Down Bad" by Taylor Swift...
Done.
Wrote Taylor-Swift_The-Tortured-Poets-Department-The-Anthology/Down-Bad.txt.
Searching for "So Long, London" by Taylor Swift...
Done.
Wrote Taylor-Swift_The-Tortured-Poets-Department-The-Anthology/So-Long,-London.txt.
Searching for "But Daddy I Love Him" by Taylor Swift...
Done.
Wrote Taylor-Swift_The-Tortured-Poets-Department-The-Anthology/But-Daddy-I-Love-Him.txt.
Searching for "Fresh Out The Slammer" by Taylor Swift...
Done.
Wrote Taylor-Swift_The-Tortured-Poets

In [21]:
# function to clean text data
def clean_text(text):
    # remove ad
    # text = text.replace('See Taylor Swift LiveGet tickets as low as $270', ' ')
    # remove ending text with number + Embed
    text = re.sub('\d*Embed', ' ', text)
    # make all text lowercase
    text = text.lower()
    for punctuation in string.punctuation:
        if punctuation == '-':
            # replace dashes ('-') with spaces
            text = text.replace(punctuation, ' ')
        else:
            # remove all other punctuation
            text = text.replace(punctuation, '')
    return text

In [22]:
# folder names with the lyric data from Genius
directory_paths = ['Taylor-Swift_Taylor-Swift/',
                   'Taylor-Swift_Fearless-Platinum-Edition/',
                   'Taylor-Swift_Fearless-Taylors-Version/',
                   'Taylor-Swift_Speak-Now-Deluxe-Edition/',
                   'Taylor-Swift_Speak-Now-Taylors-Version/',
                   'Taylor-Swift_Red-Deluxe-Edition/',
                   'Taylor-Swift_Red-Taylors-Version/',
                   'Taylor-Swift_1989-Deluxe/',
                   'Taylor-Swift_1989-Taylors-Version/',
                   'Taylor-Swift_Reputation/',
                   'Taylor-Swift_Lover/',
                   'Taylor-Swift_folklore-deluxe-version/',
                   'Taylor-Swift_evermore-deluxe-version/',
                   'Taylor-Swift_Midnights-The-Til-Dawn-Edition/',
                   'Taylor-Swift_The-Tortured-Poets-Department-The-Anthology/'
                   ]

In [94]:
pd.options.display.max_rows = 500
pd.set_option('display.max_colwidth', 0)

# create a dataframe for album title, song names, and clean lyrics
tswift_lyrics = pd.DataFrame({'Album': [], 'Song Name': [], 'Lyrics': []})

idx = 0
for i, album in enumerate(directory_paths):  
    album_name = album[13:-1].replace('-', ' ')
    for song in Path(album).glob('*.txt'):

        # remove dashes, underscores, backslashes, and '.txt' from song_name
        song_name = str(song).replace('-', ' ').replace('_', ' ').replace('\\', ' ').split('/')[0][:-4]
        # remove artist name and album name from song_name
        song_name = song_name.replace('Taylor Swift', '').replace(str(album_name), '')

        full_text = open(song, encoding='utf-8')
        lyrics_list = full_text.readlines()[1:]
        lyrics_list = [l.replace('\n', '') for l in lyrics_list]
        lyrics = ' '.join(lyrics_list)
        lyrics = clean_text(lyrics) # call clean_text function
        full_text.close()
        
        tswift_lyrics.loc[idx] = [album_name, song_name, lyrics]
        idx += 1

In [95]:
tswift_lyrics.head(10)

,Album,Song Name,Lyrics
0,Taylor Swift,A Perfectly Good Heart,a perfectly good heart why would you wanna take our love and tear it all apart now why would you wanna make the very first scar why would you wanna break a perfectly good heart maybe i shouldve seen the signs shouldve read the writing on the wall and realized by the distance in your eyes that i would be the one to fall no matter what you say i still cant believe that you would walk away it dont make sense to me but why would you wanna break a perfectly good heart why would you wanna take our love and tear it all apart now why would you wanna make the very first scar why would you wanna break a perfectly good heart its not unbroken anymore its not unbroken anymore how do i get it back the way it was before why would you wanna break a perfectly good heart why would you wanna take our love and tear it all apart now why would you wanna make the very first scar why would you wanna break— why would you wanna break it why would you wanna break a perfectly good heart why would you wanna take our love and tear it all apart now why would you wanna make the very first scar why would you wanna break a perfectly good heart mmm hmm
1,Taylor Swift,A Place In This World,cause im still trying to figure it out dont know whats down this road im just walking trying to see through the rain coming down even though im not the only one who feels the way i do im alone on my own and thats all i know ill be strong ill be wrong oh but life goes on oh im just a girl trying to find a place in this world got the radio on my old blue jeans and im wearing my heart on my sleeve feeling lucky today got the sunshine could you tell me what more do i need and tomorrows just a mystery oh yeah but thats okay im alone on my own and thats all i know ill be strong ill be wrong oh but life goes on oh im just a girl trying to find a place in this world maybe im just a girl on a mission but im ready to fly im alone on my own and thats all i know ill be strong ill be wrong oh but life goes on oh im alone on my own and thats all i know oh im just a girl trying to find a place in this world oh im just a girl oh im just a girl oh oh oh im just a girl
2,Taylor Swift,Cold as You,and when you take you take the very best of me so i start a fight cause i need to feel something and you do what you want cause im not what you wanted oh what a shame what a rainy ending given to a perfect day just walk away aint no use defending words that you will never say and now that im sitting here thinking it through ive never been anywhere cold as you you put up walls and paint them all a shade of gray and i stood there loving you and wished them all away and you come away with a great little story of a mess of a dreamer with the nerve to adore you oh what a shame what a rainy ending given to a perfect day so just walk away aint no use defending words that you will never say and now that im sitting here thinking it through ive never been anywhere cold as you you never did give a damn thing honey but i cried cried for you and i know you wouldnt have told nobody if i died died for you died for you oh what a shame what a rainy ending given to a perfect day oh every smile you fake is so condescending counting all the scars you made and now that im sitting here thinking it through ive never been anywhere cold as you ooh oh
3,Taylor Swift,I'm Only Me When I'm With You,in a field behind your yard you and i are painting pictures in the sky and sometimes we dont say a thing just listen to the crickets sing everything i need is right here by my side and i know everything about you i dont wanna live without you im only up when youre not down dont wanna fly if youre still on the ground its like no matter what i do well you drive me crazy half the time the other half im only tryin to let you know that what i feel is true and im only me when im with you just a small town boy and girl livin in a crazy world tryna figure out what is and isnt true and

In [96]:
# export lyrics as csv file
tswift_lyrics.to_csv('tswift_lyrics_data.csv', index=False)